In [1]:
# imports
import requests
import pandas as pd
import time
import os





# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [8]:
import pandas as pd
import os
import requests
from collections import Counter

# Load saved bike station data
df_bicing = pd.read_csv("bicing.csv")

# Set up Foursquare API key and headers
foursquare_key = os.getenv("FOURSQUARE_API_KEY")

headers = {
    "Accept": "application/json",
    "Authorization": foursquare_key
}

# Store both full responses and category summaries
foursquare_responses = []
category_summaries = []

for idx, row in df_bicing.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    url = f"https://api.foursquare.com/v3/places/search?ll={lat},{lon}&radius=1000&limit=50"
    
    request = requests.get(url, headers=headers)
    
    if request.status_code == 200:
        json_data = request.json()
        foursquare_responses.append(json_data)

        # Count POI categories for this station
        categories = []
        for place in json_data.get('results', []):
            for cat in place.get('categories', []):
                categories.append(cat['name'].lower())
        
        category_counts = Counter(categories)
        category_summaries.append(category_counts)
    
    else:
        print(f"Issue at index {idx}: {request.status_code}")
        foursquare_responses.append({})
        category_summaries.append({})


In [11]:
# save responses
import json

# Save full responses
with open("foursquare_responses.json", "w") as f:
    json.dump(foursquare_responses, f)

# Save category summaries
with open("category_summaries.json", "w") as f:
    json.dump([dict(c) for c in category_summaries], f)



Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [12]:
# Lists to build df_poi
poi_station_ids = []
poi_names = []
poi_latitudes = []
poi_longitudes = []
poi_locations = []
poi_categories = []
poi_distances = []

# Loop through each Foursquare response + its index (matches station index)
for station_idx, response in enumerate(foursquare_responses):
    if 'results' in response:
        for place in response['results']:
            name = place.get('name', 'N/A')
            location = place.get('location', {})
            lat = place.get('geocodes', {}).get('main', {}).get('latitude', None)
            lon = place.get('geocodes', {}).get('main', {}).get('longitude', None)
            categories = place.get('categories', [])
            category = categories[0]['name'] if categories else 'Unknown'
            distance = place.get('distance', None)

            # Add station_id to keep track
            poi_station_ids.append(station_idx)
            poi_names.append(name)
            poi_locations.append(location)
            poi_latitudes.append(lat)
            poi_longitudes.append(lon)
            poi_categories.append(category)
            poi_distances.append(distance)

            poi_category_summary = [category_summaries[idx] for idx in poi_station_ids]




Put your parsed results into a DataFrame

In [14]:

df_poi = pd.DataFrame({
    "station_index": poi_station_ids,
    "poi_name": poi_names,
    "poi_latitude": poi_latitudes,
    "poi_longitude": poi_longitudes,
    "poi_location": poi_locations,
    "poi_category": poi_categories,
    "poi_distance": poi_distances,
    "station_category_summary": poi_category_summary
})


print(df_poi.head())




   station_index              poi_name  poi_latitude  poi_longitude  \
0              0        Vermuteria Lou     41.405910       2.162397   
1              0       Taverna el Glop     41.404895       2.159547   
2              0            La Catrina     41.407947       2.159362   
3              0  La Antigua de Mexico     41.403826       2.158413   
4              0            Verdi Park     41.404424       2.157652   

                                        poi_location  \
0  {'address': 'Carrer Escorial, 3', 'address_ext...   
1  {'address': 'Sant Lluís, 24', 'admin_region': ...   
2  {'address': 'Providencia, 88', 'admin_region':...   
3  {'address': 'Torrijos, 50', 'admin_region': 'C...   
4  {'address': 'Calle Torrijos, 49', 'admin_regio...   

               poi_category  poi_distance  \
0                       Bar            48   
1                 BBQ Joint           237   
2  Mediterranean Restaurant           361   
3           Taco Restaurant           371   
4       Ind

In [ ]:
df_poi.to_csv('poi.csv')

In [ ]:
df_poi.isnull().sum()


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating